In [1]:
import uproot

import pandas as pd
import numpy as np
import time
#import numba
import matplotlib.pyplot as plt

In [2]:
from encode import encode, decode

encodeList = np.vectorize(encode)

In [3]:
def getElinkInput(df):
    linkData = []
    for i in range(12):
        thislinkData = ''
        for j in range(4):
            val = df[(df.eLink==i) & (df.eLinkChannel==j)].encodedCharge.values
            if len(val)==0:
                thislinkData += '0000000'
            else:
                thislinkData += format(int(val[0]), '#0%ib'%(9))[2:]
        linkData.append(thislinkData)
    return linkData

In [4]:
###UPDATED TO USE CSV OUTPUTS
def getAlgoBlockOutputBestChoice(valscDF, nTC, sumEncoding=(0,4,4)):
    df = pd.DataFrame(valscDF,columns =['calibCharge'])
    SUM = encode(df.calibCharge.sum().astype(np.int),sumEncoding[0], sumEncoding[1], sumEncoding[2],asInt=True)
    TOPNCHARGE = (df.nlargest(nTC,'calibCharge'))
    selidx = TOPNCHARGE.index
    noselidx = list(set(df.index)^set(TOPNCHARGE.index))
    df = df.copy(deep=True)
    df.iloc[noselidx, df.columns.get_loc('calibCharge')] = 0
    ADD_MAP = (df.calibCharge>0).values.astype(np.int)
#    CHARGES = [x for x in df.calibCharge if x != 0]  ### if charges not encoded
    CHARGES = [encode(x,3, 4, 3,asInt=True) for x in df.calibCharge if x != 0]  ### if charges encoded
#    print (ADD_MAP,CHARGES)
    return (ADD_MAP,CHARGES,SUM)


def getAlgoBlockOutputThreshold(tc_Charge, thresholds = np.array([0]*48), sumEncoding=(0,5,3), tcEncoding=(0,4,4)):
    MOD_SUM_0 = encode(tc_Charge[0:16].sum(),sumEncoding[0], sumEncoding[1], sumEncoding[2],asInt=True)
    MOD_SUM_1 = encode(tc_Charge[16:32].sum(),sumEncoding[0], sumEncoding[1], sumEncoding[2],asInt=True)
    MOD_SUM_2 = encode(tc_Charge[32:48].sum(),sumEncoding[0], sumEncoding[1], sumEncoding[2],asInt=True)
    
    SUM = encode(tc_Charge.sum(),sumEncoding[0], sumEncoding[1], sumEncoding[2],asInt=True)
    pass_threshold = tc_Charge >= thresholds    
    NTCQ = pass_threshold.sum()
    ADD_MAP = pass_threshold
    ADD_MAP.dtype=np.int8
    
    CHARGEQ = encodeList(tc_Charge[pass_threshold],tcEncoding[0], tcEncoding[1],tcEncoding[2], False,True)

    return (NTCQ, ADD_MAP, CHARGEQ,SUM,MOD_SUM_0,MOD_SUM_1,MOD_SUM_2)



def bitmapArrayToInt(bitmap):
    total = 0
    for i in range(len(bitmap)):
        total += bitmap[i]<<i
    return total, format(total, '#0%ib'%(len(bitmap)+2))[2:]

In [5]:
def formatBestChoiceOutput(BC, NTCQ, ADD_MAP, CHARGEQ,SUM):
    header = format(BC, '#0%ib'%(15))[-5:]
    dataType = ''
    modSumData = format(SUM, '#0%ib'%(10))[2:]

    if NTCQ<8:
        bitmap = np.array([int(x) for x in format(ADD_MAP, '#0%ib'%(48+2))[2:]][::-1])
        channelNumbers = np.arange(48)[bitmap==1]
        channelNumbersBin = [format(x,'#0%ib'%(6+2))[2:] for x in channelNumbers]
        AddressMapData = ''
        for x in channelNumbersBin: AddressMapData += x
       
        ChargeData = ''
        for x in CHARGEQ:
            ChargeData+=format(x, '#0%ib'%(9))[2:]
        
    else:

        AddressMapData=format(ADD_MAP, '#0%ib'%(48+2))[2:]
        ChargeData = ''
        for x in CHARGEQ:
            ChargeData+=format(x, '#0%ib'%(9))[2:]


    formattedData = header + modSumData +  AddressMapData + ChargeData

    if len(formattedData)%16==0:
        nPadBits=0
        paddedData = formattedData
    else:
        nPadBits = 16 - (len(formattedData)%16)
#        print ("nPadBits",nPadBits)
        paddedData = formattedData + '0'*nPadBits


    return paddedData

def formatThresholdOutput(BC, NTCQ, ADD_MAP, CHARGEQ,SUM,MOD_SUM_0,MOD_SUM_1,MOD_SUM_2):
    header = format(BC, '#0%ib'%(15))[-5:]
    dataType = ''
    if NTCQ==0: 
        dataType='000'
    elif NTCQ<8:
        dataType='001'
    else:
        dataType='010'
    modSumData = format(SUM, '#0%ib'%(10))[2:]
    extraBit=''
    if NTCQ==0:
        nChannelData=''
        AddressMapData=''
        ChargeData=''
    elif NTCQ<8:
        nChannelData=format(NTCQ, '#0%ib'%(3+2))[2:]
        
        bitmap = np.array([int(x) for x in format(ADD_MAP, '#0%ib'%(48+2))[2:]][::-1])
        channelNumbers = np.arange(48)[bitmap==1]
        channelNumbersBin = [format(x,'#0%ib'%(6+2))[2:] for x in channelNumbers]
        AddressMapData = ''
        for x in channelNumbersBin: AddressMapData += x
        
        ChargeData = ''
        for x in CHARGEQ:
            ChargeData+=format(x, '#0%ib'%(9))[2:]
        
    else:
        nChannelData=''
        AddressMapData=format(ADD_MAP, '#0%ib'%(48+2))[2:]
        ChargeData = ''
        for x in CHARGEQ:
            ChargeData+=format(x, '#0%ib'%(9))[2:]

    formattedData = header + dataType + modSumData + extraBit + nChannelData + AddressMapData + ChargeData
    if len(formattedData)%16==0:
        nPadBits=0
        paddedData = formattedData
    else:
        nPadBits = 16 - (len(formattedData)%16)
        paddedData = formattedData + '0'*nPadBits

#    print(NTCQ,nPadBits)
    
    return paddedData

In [6]:
def formatThresholdOutput_Truncated1(BC, NTCQ, ADD_MAP, CHARGEQ,SUM,MOD_SUM_0,MOD_SUM_1,MOD_SUM_2):
    header = format(BC, '#0%ib'%(15))[-5:]
    dataType = '110'
    modSumData = format(SUM, '#0%ib'%(10))[2:]

    formattedData = header + dataType + modSumData
    return formattedData

def formatThresholdOutput_Truncated2(BC, NTCQ, ADD_MAP, CHARGEQ,SUM,MOD_SUM_0,MOD_SUM_1,MOD_SUM_2):
    header = format(BC, '#0%ib'%(15))[-5:]
    dataType = '110'
    modSumData = format(MOD_SUM_0, '#0%ib'%(10))[2:] + format(MOD_SUM_1, '#0%ib'%(10))[2:] + format(MOD_SUM_2, '#0%ib'%(10))[2:]

    formattedData = header + dataType + modSumData
    return formattedData


In [7]:
def calibRegToCorr(register):
    ### bits
    ### 5,    4,    3,    2,    1,    0
    ### 1,  1/2,  1/4,  1/8, 1/16, 1/32

    corr = 0.
    corr += 1./32 * (register>>0&1)
    corr += 1./16 * (register>>1&1)
    corr += 1./8  * (register>>2&1)
    corr += 1./4  * (register>>3&1)
    corr += 1./2  * (register>>4&1)
    corr +=  1.   * (register>>5&1)
#    corr = 1. + corr
    return corr
calibRegToCorr = np.vectorize(calibRegToCorr)

calibRegisters = np.ones(48).astype(np.int)
calibRegisters = np.random.choice([28,29,30,31,32,33,34,35,36,37],48)

corrFactor= calibRegToCorr(calibRegisters)
    

In [8]:
####Selection of a single wafer
subdet=3
layer=5
wafer = 31

In [9]:
tcmapCSVname = 'TC_ELINK_MAP.csv'
df_tcmap = pd.read_csv(tcmapCSVname)

In [10]:
fName='root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/NewNtuples/ntuple_hgcalNtuples_vbf_hmm_200PU_1.root'
_tree = uproot.open(fName,xrootdsource=dict(chunkbytes=1024**3, limitbytes=1024**3))['hgcalTriggerNtuplizer/HGCalTriggerNtuple']

In [11]:
#load dataframe
#df = _tree.pandas.df( ['tc_subdet','tc_zside','tc_layer','tc_wafer','tc_cell','tc_uncompressedCharge','tc_compressedCharge','tc_data','tc_mipPt'],entrystart=0,entrystop=25)
df = _tree.pandas.df( ['tc_subdet','tc_zside','tc_layer','tc_wafer','tc_cell','tc_uncompressedCharge','tc_compressedCharge','tc_data','tc_mipPt'])
df.columns = ['subdet','zside','layer','wafer','triggercell','uncompressedCharge','compressedCharge','data','mipPt']

#remove unwanted layers
df = df[(df.subdet==subdet) & (df.layer==layer) & (df.wafer==wafer)]

In [12]:
#set index
df.set_index(['subdet','zside','layer','wafer','triggercell'],append=True,inplace=True)
df.reset_index('subentry',drop=True,inplace=True)
df.sort_index(inplace=True)


In [13]:
#split +/- zside into separate entries
df.reset_index(inplace=True)
maxN = df.entry.max()
df.set_index(['zside'],inplace=True)

df.loc[-1,['entry']] = df.loc[-1,['entry']] + maxN+1
df.reset_index(inplace=True)
df.set_index(['entry','subdet','zside','layer','wafer','triggercell'],inplace=True)


In [14]:
#get geometry information
geomName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/triggerGeomV9.root"
geomTree = uproot.open(geomName,xrootdsource=dict(chunkbytes=1024**3, limitbytes=1024**3))["hgcaltriggergeomtester/TreeTriggerCells"]


In [15]:
geomDF = geomTree.pandas.df(['subdet','zside','layer','wafer','triggercell','x','y','z','c_n'])
geomDF['r'] = (geomDF['x']**2 + geomDF['y']**2)**.5
geomDF['eta'] = np.arcsinh(geomDF.z/geomDF.r)
geomDF.set_index(['subdet','zside','layer','wafer','triggercell'],inplace=True)
geomDF['isHDM'] = geomDF.c_n>4
geomDF.sort_index(inplace=True)
geomDF.drop(['x','y','c_n'],axis=1,inplace=True)

In [16]:
#### Need to update layer list in geomdf for subdet 4 and 5 to match df
geomDF.reset_index(inplace=True)
geomDF.loc[geomDF.subdet==4,'layer'] += 28
geomDF.loc[geomDF.subdet==5,'layer'] += 28
geomDF.set_index(['subdet','zside','layer','wafer','triggercell'],inplace=True)


In [17]:
threshold_mipPt = 1.35
fCtoADC = 100./1024.
geomDF['threshold_fC'] = threshold_mipPt*np.cosh(geomDF.eta) *3.43
geomDF['threshold_ADC'] = np.round(geomDF.threshold_fC/fCtoADC).astype(np.int)
#geomDF['threshold_ADC'] = geomDF.threshold_fC/fCtoADC


In [18]:
df.reset_index('entry',inplace=True)

df['isHDM'] = geomDF['isHDM']
# df['eta'] = geomDF['eta']
df['threshold_ADC'] = geomDF['threshold_ADC']
#df['threshold_ADC_int'] = geomDF['threshold_ADC_int']

df.reset_index(inplace=True)
df.set_index(['entry','subdet','zside','layer','wafer','triggercell'],inplace=True)

In [19]:
nExp = 4
nMant = 3
nDropHDM = 3
nDropLDM = 1
roundBits = False

df['encodedCharge'] = np.where(df.isHDM,df.uncompressedCharge.apply(encode,args=(nDropHDM,nExp,nMant,roundBits,True)),df.uncompressedCharge.apply(encode,args=(nDropLDM,nExp,nMant,roundBits,True)))
df['decodedCharge'] = np.where(df.isHDM,df.encodedCharge.apply(decode,args=(nDropHDM,nExp,nMant)),df.encodedCharge.apply(decode,args=(nDropLDM,nExp,nMant)))
df['pass_135'] = df.decodedCharge>df.threshold_ADC

In [20]:
df.reset_index('zside',drop=True,inplace=True)

In [21]:
singleEvent = geomDF.reset_index()
singleEvent = singleEvent[(singleEvent.subdet==subdet) & (singleEvent.zside==1) & (singleEvent.layer==layer) & (singleEvent.wafer==wafer)]
#geomDF = geomDF[(geomDF.subdet==subdet) & (geomDF.zside==1) & (geomDF.layer==layer) & (geomDF.wafer==wafer)]
singleEvent.set_index(['subdet','layer','wafer','triggercell'],inplace=True)

In [22]:
#singleEvent.reset_index(inplace=True)
singleEvent = singleEvent.reset_index().merge(df_tcmap, on=['triggercell', 'isHDM']).set_index(['subdet','layer','wafer','triggercell'])
#singleEvent.set_index(['subdet','layer','wafer','triggercell'],inplace=True)


In [23]:
#singleEvent

In [24]:
###Input csv
inputLines = "OrbitNumber, BX, "
for i in range(12):
    inputLines += "ePortRxDataGroup_%i, "%i
for i in range(48):
    inputLines += "F2F_%i, "%i
for i in range(48):
    inputLines += "CALQ_%i, "%i

inputLines = inputLines[:-2]+"\n"

eventList = df.reset_index().entry.unique()
eventTracker = []

for eNum in range(100):
#for eNum in range(1):    
    event = np.random.choice(eventList)
    eventTracker.append(event)

    OrbitNumber = int(eNum/3564)
    BX = eNum % 3564
    
    singleEvent['encodedCharge'] = df.loc[event]['encodedCharge']
    singleEvent['decodedCharge'] = df.loc[event]['decodedCharge']
    singleEvent.encodedCharge.fillna(0,inplace=True)
    singleEvent.decodedCharge.fillna(0,inplace=True)

    singleEvent['calibCharge'] = (singleEvent.decodedCharge * corrFactor).round().astype(np.int)
    singleEvent['calibCharge_Encoded'] = np.where(singleEvent.isHDM,singleEvent.calibCharge.apply(encode,args=(nDropHDM,nExp,nMant,True,True)),singleEvent.calibCharge.apply(encode,args=(nDropLDM,nExp,nMant,True,True)))
    singleEvent['pass_135'] = singleEvent.calibCharge>=singleEvent.threshold_ADC
    
    inputLines += "%i, %i, "%(OrbitNumber, BX)
    
    for x in getElinkInput(singleEvent):
        inputLines += "%s, "%x

    for x in singleEvent.decodedCharge.values:
        inputLines += "%i, "%x

    for x in singleEvent.calibCharge.values:
        inputLines += "%i, "%x
    inputLines = inputLines[:-2]+"\n"
    
#print (inputLines)

In [25]:
with open('CSV_Files/Inputs.csv','w') as _file:
    _file.write(inputLines)

In [26]:
algoInputData = pd.read_csv('CSV_Files/Inputs.csv',index_col=['OrbitNumber','BX'],skipinitialspace=True)
#algoInputData.loc[0,0]

In [27]:
eLinkInputLines = "BX, "

calibInputLines = "BX, "

algoInputLines = "BX, "

algoOutputLines = ""
algoOutputLines += "BX, NTCQ, ADD_MAP, "

for i in range(12):
    eLinkInputLines += "ePortRxDataGroup_%i, "%i
for i in range(48):
    calibInputLines += "SM_%i, "%i
    algoInputLines += "CALQ_%i, "%i
    algoOutputLines += "CHARGEQ_%i, "%i

eLinkInputLines = eLinkInputLines[:-2]+"\n"

calibInputLines = calibInputLines[:-2]+"\n"
algoInputLines = algoInputLines[:-2]+"\n"
algoOutputLines+="SUM, MOD_SUM_0, MOD_SUM_1, MOD_SUM_2\n"

formattedDataLines = 'BX, DATAHEX, DATA\n'
formattedDataLines_Truncated1 = 'BX, DATAHEX, DATA\n'
formattedDataLines_Truncated2 = 'BX, DATAHEX, DATA\n'

eventList = df.reset_index().entry.unique()
eventTracker = []


cols = [x for x in algoInputData.columns.tolist() if 'CALQ' in x]
algoInputData[cols].loc[0,0]

colsi = [x for x in algoInputData.columns.tolist() if 'ePortRxDataGroup' in x]
colsdec = [x for x in algoInputData.columns.tolist() if 'F2F' in x]
colsc = [x for x in algoInputData.columns.tolist() if 'CALQ' in x]

#thresholds = np.array([0]*48)
thresholds = singleEvent.threshold_ADC.values
isHDM = singleEvent.isHDM.any()

#print (thresholds)
#print (isHDM)


nLayer_nLink = {1  : 2,
                  2  : 2,
                  3  : 2,
                  4  : 2,
                  5  : 2,
                  6  : 2,
                  7  : 4,
                  8  : 2,
                  9  : 5,
                  10 : 2,
                  11 : 4,
                  12 : 2,
                  13 : 3,
                  14 : 2,
                  15 : 2,
                  16 : 2,
                  17 : 2,
                  18 : 2,
                  19 : 2,
                  20 : 2,
                  21 : 2,
                  22 : 2,
                  23 : 2,
                  24 : 2,
                  25 : 2,
                  26 : 2,
                  27 : 2,
                  28 : 2,
                  29 : 2,
                  30 : 2,
                  31 : 2,
                  32 : 2,
                  33 : 2,
                  34 : 2,
                  35 : 2,
                  36 : 2,
                  37 : 2,
                  38 : 2,
                  39 : 2,
                  40 : 2,
                  41 : 2,
                  42 : 2,
                  43 : 2,
                  44 : 2,
                  45 : 2,
                  46 : 2,
                  47 : 2,
                  48 : 2,
                  49 : 2,
                  50 : 2,
                  51 : 2,
                  52 : 2}
    
nLink_nTC = {0:0, 1:1, 2:3, 3:6, 4:9, 5:14, 6:18, 7:23, 8:27, 9:32, 10:37, 11:41, 12:46}
nTC_bestchoice = nLink_nTC[nLayer_nLink[layer]]
algoOutputLinesBestChoice = "BX, "
for i in range(nTC_bestchoice):
    algoOutputLinesBestChoice += "CHARGE_%i, "%i
algoOutputLinesBestChoice += "ADD_MAP, SUM\n"
formattedDataLinesBestChoice = 'BX, DATA\n'

for BX in range(algoInputData.shape[0]):
#for BX in range(1):    

############### input stuff
    eLinkInputLines += "%i, "%BX
    valsi = algoInputData.loc[0,BX][colsi].values
    for x in valsi:
        eLinkInputLines += "%s, "%x
    eLinkInputLines = eLinkInputLines[:-2]+"\n"
    
    calibInputLines += "%i, "%BX
    valsd = algoInputData.loc[0,BX][colsdec].values
    for x in valsd:
        calibInputLines += "%i, "%x
    calibInputLines = calibInputLines[:-2]+"\n"
    

    algoInputLines += "%i, "%BX
    valsc = algoInputData.loc[0,BX][colsc].values
    for x in valsc:
        algoInputLines += "%i, "%x
    algoInputLines = algoInputLines[:-2]+"\n"
    

############### threshold
    tcCharge = valsc

#    passThresh = tcCharge > thresholds
    tcEncoding = (3,4,3) if isHDM else (1,4,3)
    
    algoOutputLines += "%i, "%BX
    NTCQ, ADD_MAP, CHARGEQ,SUM,MOD_SUM_0,MOD_SUM_1,MOD_SUM_2 = getAlgoBlockOutputThreshold(tcCharge, thresholds, sumEncoding=(0,5,3),tcEncoding=tcEncoding)
    ADD_MAP_INT = bitmapArrayToInt(ADD_MAP)[0]
    algoOutputLines += "%i, %i, "%(NTCQ, ADD_MAP_INT)
    for x in CHARGEQ:
        algoOutputLines += "%i, "%x
    for i in range(48-len(CHARGEQ)):
        algoOutputLines += "0, "
    algoOutputLines += "%i, %i, %i, %i\n"%( SUM, MOD_SUM_0, MOD_SUM_1, MOD_SUM_2)

    data = formatThresholdOutput(BX, NTCQ, ADD_MAP_INT, CHARGEQ,SUM,MOD_SUM_0,MOD_SUM_1,MOD_SUM_2)
    dataTruncated1 = formatThresholdOutput_Truncated1(BX, NTCQ, ADD_MAP_INT, CHARGEQ,SUM,MOD_SUM_0,MOD_SUM_1,MOD_SUM_2)
    dataTruncated2 = formatThresholdOutput_Truncated2(BX, NTCQ, ADD_MAP_INT, CHARGEQ,SUM,MOD_SUM_0,MOD_SUM_1,MOD_SUM_2)
    

    formattedDataLines += "%i, "%BX
    formattedDataLines_Truncated1 += "%i, "%BX
    formattedDataLines_Truncated2 += "%i, "%BX
    formattedDataLines += format(int(data,2), '#0%ix'%(len(data)/4+2)) + ", "+data+"\n"
    formattedDataLines_Truncated1 += format(int(dataTruncated1,2), '#0%ix'%(len(dataTruncated1)/4+2)) + ", "+dataTruncated1+"\n"
    formattedDataLines_Truncated2 += format(int(dataTruncated2,2), '#0%ix'%(len(dataTruncated2)/4+2)) + ", "+dataTruncated2+"\n"

############### best choice
     
    valscDF = algoInputData[colsc].loc[0,BX].values
#    print (valscDF)
    
#    print (dfBC)

    formattedDataLinesBestChoice += "%i, "%BX
    
    ADD_MAP_BC ,CHARGES_BC ,SUM_BC = getAlgoBlockOutputBestChoice(valscDF, nTC_bestchoice)
    ADD_MAP_INT_BC = bitmapArrayToInt(ADD_MAP_BC)[0]
    algoOutputLinesBestChoice += "%i, "%BX
    for x in CHARGES_BC:
        algoOutputLinesBestChoice += "%i, "%x
    
    algoOutputLinesBestChoice += "%i, %i \n"%(ADD_MAP_INT_BC , SUM_BC)   
    
    data = formatBestChoiceOutput(BX, nTC_bestchoice , ADD_MAP_INT_BC, CHARGES_BC,SUM_BC)
    formattedDataLinesBestChoice += data+" \n"

print (algoOutputLinesBestChoice)    
    

BX, CHARGE_0, CHARGE_1, CHARGE_2, ADD_MAP, SUM
0, 45, 45, 49, 4947802341376, 184 
1, 47, 48, 47, 12884901920, 186 
2, 36, 44, 42, 8796093022496, 169 
3, 48, 48, 60, 1103806599168, 184 
4, 42, 40, 40, 290, 179 
5, 43, 56, 44, 8589951488, 188 
6, 46, 47, 45, 8796093054980, 183 
7, 46, 42, 42, 1099511627824, 172 
8, 48, 45, 50, 4398046576704, 185 
9, 45, 52, 43, 140737488359425, 180 
10, 49, 45, 45, 81, 180 
11, 38, 49, 41, 2199023260160, 173 
12, 40, 39, 39, 536873024, 169 
13, 45, 45, 42, 8590467072, 182 
14, 44, 47, 43, 44, 180 
15, 50, 39, 41, 264224, 179 
16, 40, 41, 41, 292057776136, 173 
17, 43, 56, 44, 8589951488, 188 
18, 45, 48, 48, 264200, 181 
19, 41, 41, 52, 4406636462080, 180 
20, 47, 48, 47, 12884901920, 186 
21, 41, 42, 41, 17179870272, 177 
22, 40, 39, 39, 536873024, 169 
23, 38, 49, 41, 2199023260160, 173 
24, 43, 43, 44, 4295229441, 178 
25, 45, 44, 46, 549756078080, 174 
26, 41, 40, 40, 4294984704, 174 
27, 40, 42, 41, 300647710720, 174 
28, 43, 36, 40, 1073745936, 170

In [28]:
with open('CSV_Files/eLinkInput.csv','w') as _file:
    _file.write(eLinkInputLines)

In [29]:
with open('CSV_Files/CalibBlockInput.csv','w') as _file:
    _file.write(calibInputLines)

In [30]:
with open('CSV_Files/AlgoBlockInput.csv','w') as _file:
    _file.write(algoInputLines)

In [31]:
with open('CSV_Files/AlgoBlockOutput_Threshold_135.csv','w') as _file:
    _file.write(algoOutputLines)

In [32]:
with open('CSV_Files/FormattingBlockOutput_Threshold_135.csv','w') as _file:
    _file.write(formattedDataLines)

In [33]:
with open('CSV_Files/FormattingBlockOutput_Threshold_135_Truncated1.csv','w') as _file:
    _file.write(formattedDataLines_Truncated1)

with open('CSV_Files/FormattingBlockOutput_Threshold_135_Truncated2.csv','w') as _file:
    _file.write(formattedDataLines_Truncated2)    

In [34]:
with open('CSV_Files/AlgoBlockOutput_BestChoice.csv','w') as _file:
    _file.write(algoOutputLinesBestChoice)
with open('CSV_Files/FormattingBlockOutput_BestChoice.csv','w') as _file:
    _file.write(formattedDataLinesBestChoice)

In [35]:
with open('CSV_Files/Control_Module_Info.csv','w') as _file:
    values = "SUBDET, LAYER, MODULE\n"
    values += "%i, %i, %i"%(subdet, layer, wafer)
    values += "\n\n\n"
    values += str(eventTracker)
    _file.write(values)
    
with open('CSV_Files/Control_Threshold_Levels.csv','w') as _file:
    threshold_values = str(singleEvent.threshold_ADC.values.tolist())
    threshold_values = threshold_values[1:-1]
    line = ''
    for i in range(48):
        line += 'THRESHV_%i, '%i
    line = line[:-2] + '\n'
    line += threshold_values + '\n'
    _file.write(line)

    
with open('CSV_Files/Control_Calibration_Values.csv','w') as _file:
    calib_values = str(calibRegisters.tolist())
    calib_values = calib_values[1:-1]
    line = ''
    for i in range(48):
        line += 'CALV_%i, '%i
    line = line[:-2] + '\n'
    line += calib_values + '\n'
    _file.write(line)

with open('CSV_Files/Control_GeneralInformation.csv','w') as _file:
    line = "N_INPUT_LINKS, HDM\n"
    line += '12, 1\n'
    _file.write(line)


In [36]:
fileName = "FormattingBlockOutput_Threshold_135"
inputFileName = "CSV_Files/%s.csv"%fileName
inputFileNameTruncate1 = "CSV_Files/%s_Truncated1.csv"%fileName
inputFileNameTruncate2 = "CSV_Files/%s_Truncated2.csv"%fileName


In [38]:
Bufferdf = pd.read_csv(inputFileName,index_col='BX',skipinitialspace=True)
Bufferdf_1 = pd.read_csv(inputFileNameTruncate1,index_col='BX',skipinitialspace=True)
Bufferdf_2 = pd.read_csv(inputFileNameTruncate2,index_col='BX',skipinitialspace=True)

In [39]:

totalBufferSize = 840 ### in 16-bit wordss

def getBufferOutputs(nOutputLinks, nBitsPerLink=32, maxLatency=12):

    T2 = maxLatency*nOutputLinks*2
    T1 = T2 - 25 ###max data size, 25 16-bit words

    BufferContents = ""

    line = "BC, BufferSize, CurrentDataSize, T1, T2, Truncated1word, Truncated2words"
    for i in range(nOutputLinks):
        line += ", eLink_%i"%i
    line += "\n"


    for BC in Bufferdf.index:
        currentBXData = Bufferdf.loc[BC].DATA
        trun1 = False
        trun2 = False
        bufferSize = len(BufferContents)/16
        currentDataSize = len(currentBXData)/16
    #     print (bufferSize, currentDataSize)
    #     print (BufferContents)
        if (bufferSize + currentDataSize)>T2:
            currentBXData = Bufferdf_1.loc[BC].DATA        
            trun1 = True
        elif (len(BufferContents)/16)>T1:
            currentBXData = Bufferdf_2.loc[BC].DATA
            trun2 = True
        else:
            BufferContents += Bufferdf.loc[BC].DATA

        line += "%i, %i, %i, %i, %i, %i, %i, "%(BC, bufferSize, currentDataSize, T1, T2, trun1, trun2)

        buffers = []
        for i in range(nOutputLinks):
            linkData = BufferContents[:nBitsPerLink]
            BufferContents = BufferContents[nBitsPerLink:]
            nBits = len(linkData)
            if nBits<nBitsPerLink:
                linkData = '0'*(nBitsPerLink-nBits) + linkData
            dataHex = format(int(linkData,2), '#0%ix'%(len(linkData)/4+2))
            buffers.append(format(int(linkData,2), '#0%ix'%(len(linkData)/4+2)))
            line += "%s, "%dataHex
        line = line[:-2]+"\n"
    #    print (buffers, len(BufferContents))
    return line
        

In [40]:
nBitsPerLink = 32
maxLatency = 12

for nLinks in range(2,12):
    l = getBufferOutputs(nLinks)
    
    with open("CSV_Files/BufferOutput_Threshold_135_outputLinks_%i.csv"%nLinks,"w") as _file:
        _file.write(l)